# YouTube video
## Loading documents from a YouTube url
- https://python.langchain.com/docs/integrations/document_loaders/youtube_audio

### ffprobe or avprobe not found. Please install one
- brew install ffmpeg
- https://stackoverflow.com/questions/30770155/ffprobe-or-avprobe-not-found-please-install-one

In [22]:
import sys
!{sys.executable} -m pip install yt_dlp
!{sys.executable} -m pip install pydub
!{sys.executable} -m pip install tiktoken
!{sys.executable} -m pip install faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 17.2 MB/s eta 0:00:0000:0100:01


In [9]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']


- Use YoutubeAudioLoader to fetch / download the audio files.
- Then, ues OpenAIWhisperParser() to transcribe them to text.
- We will use yt_dlp to download audio for YouTube urls.
- We will use pydub to split downloaded audio files (such that we adhere to Whisper API's 25MB file size limit).

In [10]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader


In [11]:
url="https://www.youtube.com/watch?v=NMAcu-WS9nE"
save_dir="docs/youtube/"
loader = GenericLoader(
    YoutubeAudioLoader([url],save_dir),
    OpenAIWhisperParser()
)
docs = loader.load()


[youtube] Extracting URL: https://www.youtube.com/watch?v=NMAcu-WS9nE
[youtube] NMAcu-WS9nE: Downloading webpage
[youtube] NMAcu-WS9nE: Downloading ios player API JSON
[youtube] NMAcu-WS9nE: Downloading android player API JSON
[youtube] NMAcu-WS9nE: Downloading m3u8 information
[info] NMAcu-WS9nE: Downloading 1 format(s): 140
[download] docs/youtube//『原來。爸爸一直都在』記得和爸爸說聲父親節快樂｜緯來電視.m4a has already been downloaded
[download] 100% of    4.00MiB
[ExtractAudio] Not converting audio docs/youtube//『原來。爸爸一直都在』記得和爸爸說聲父親節快樂｜緯來電視.m4a; file is already in target format m4a
Transcribing part 1!


In [12]:
len(docs)


1

In [13]:
docs[0]


Document(page_content='每次回家,都覺得門好重。 我不太記得上次跟我爸講話是什麼時候了。 從小,我對爸爸的記憶就是 不是在屋裡看他, 就是看他關門。 爸爸今天學校運動會。 或是,根本沒有他。 或是,他根本沒在乎過我。 這麼晚了,還在處理公司的事。 沒有啦,事關大事而已。 不要想太多啦。 小雨啊,去爸爸的房間幫他擦乾淨。 小雨啊,去爸爸的房間幫他擦乾淨。 小雨啊,去爸爸的房間幫他擦乾淨。 小雨啊,去爸爸的房間幫他擦乾淨。 小雨啊,去爸爸的房間幫他擦乾淨。 小雨啊,去爸爸的房間幫他擦乾淨。 小雨啊,去爸爸的房間幫他擦乾淨。 小雨啊,去爸爸的房間幫他擦乾淨。 從一天到一天, 我一直想要更多。 我希望我能夠不被遺忘。 有時候我覺得, 你會說, 爸爸,父親節快樂。 爸爸,父親節快樂。 爸爸, 我們在等什麼呢? 我們在等什麼呢? 我們在等什麼呢?', metadata={'source': 'docs/youtube/『原來。爸爸一直都在』記得和爸爸說聲父親節快樂｜緯來電視.m4a', 'chunk': 0})

In [15]:
docs[0].page_content[0:200]


'每次回家,都覺得門好重。 我不太記得上次跟我爸講話是什麼時候了。 從小,我對爸爸的記憶就是 不是在屋裡看他, 就是看他關門。 爸爸今天學校運動會。 或是,根本沒有他。 或是,他根本沒在乎過我。 這麼晚了,還在處理公司的事。 沒有啦,事關大事而已。 不要想太多啦。 小雨啊,去爸爸的房間幫他擦乾淨。 小雨啊,去爸爸的房間幫他擦乾淨。 小雨啊,去爸爸的房間幫他擦乾淨。 小雨啊,去爸爸的房間幫他擦乾淨。 '

## Building a chat app from YouTube video

In [16]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [17]:
# Combine doc
combined_docs = [doc.page_content for doc in docs]
text = " ".join(combined_docs)


In [18]:
# Split them
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
splits = text_splitter.split_text(text)


In [23]:
# Build an index
embeddings = OpenAIEmbeddings()
vectordb = FAISS.from_texts(splits, embeddings)


In [24]:
# Build a QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
)


In [25]:
# Ask a question!
query = "我對爸爸的記憶是什麼?"
qa_chain.run(query)


'你對爸爸的記憶是不是在屋裡看他或者看他關門。'

In [27]:
query = "什麼時候學校運動會?"
qa_chain.run(query)


'我不知道學校運動會是什麼時候。'

In [28]:
query = "爸爸有去學校運動會嗎?"
qa_chain.run(query)


'是的，爸爸有去學校運動會。'

## Summarizing

In [30]:
def get_completion(prompt, model="gpt-3.5-turbo"): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]
    

### Summarize with a word/sentence/character limit

In [38]:
prompt = f"""
你的任務是產生一個劇情或故事的簡短摘要。

將下方被三個反引號標記的故事摘要化，最多60個字。

故事： ```{docs[0].page_content}```
"""

response = get_completion(prompt)
print(response)


每次回家，都覺得門好重。我對爸爸的記憶就是看他關門。爸爸今天學校運動會，或是根本沒有他。這麼晚了，還在處理公司的事。小雨啊，去爸爸的房間幫他擦乾淨。從一天到一天，我一直想要更多。爸爸，我們在等什麼呢？


## Summarize with a focus ...

In [39]:
prompt = f"""
你的任務是產生一個劇情或故事的簡短摘要。

將下方被三個反引號標記的故事摘要化，並專注於感人或揪心的情節或情境。限制在60個字以內。

故事： ```{docs[0].page_content}```
"""

response = get_completion(prompt)
print(response)


每次回家，都覺得門好重。我不太記得上次跟我爸講話是什麼時候了。從小，我對爸爸的記憶就是不是在屋裡看他，就是看他關門。爸爸今天學校運動會。或是，根本沒有他。或是，他根本沒在乎過我。這麼晚了，還在處理公司的事。沒有啦，事關大事而已。不要想太多啦。小雨啊，去爸爸的房間幫他擦乾淨。從一天到一天，我一直想要更多。我希望我能夠不被遺忘。有時候我覺得，你會說，爸爸，父親節快樂。爸爸，我們在等什麼呢?


## Try "extract" instead of "summarize"

In [41]:
prompt = f"""
你的任務是產生一個劇情或故事的簡短摘要。

將下方被三個反引號標記的故事中，提取與感人或揪心的情節或情境。限制在60個字以內。

故事： ```{docs[0].page_content}```
"""

response = get_completion(prompt)
print(response)


每次回家，都覺得門好重。從小，我對爸爸的記憶就是看他關門。爸爸今天學校運動會，或是根本沒有他。這麼晚了，還在處理公司的事。小雨啊，去爸爸的房間幫他擦乾淨。從一天到一天，我一直想要更多。我希望我能夠不被遺忘。有時候我覺得，爸爸，父親節快樂。我們在等什麼呢？
